In [4]:
import pandas as pd 
import networkx as nx 

In [21]:
import csv

with open("graphs/data/lastfm_asia_edges.csv","r") as f:

    edges = csv.reader(f)
    output = list(edges)[1:]


In [22]:
G = nx.Graph()

In [23]:
G.add_edges_from(output)

In [27]:
with open("graphs/data/lastfm_asia_target.csv","r") as f:

    targets = csv.reader(f)
    output_targets = list(targets)[1:]

In [31]:
output_targets[:5]

output_targets = {sublist[0]: sublist[1] for sublist in output_targets}

In [33]:
nx.set_node_attributes(G, values=output_targets, name="target")

In [34]:
nx.write_gexf(G, "test.gexf")

In [75]:
import json

with open("graphs/data/lastfm_asia_features.json",'rb') as f:

    features = json.load(f)

In [76]:
import itertools 

In [77]:
len(set(itertools.chain(*features.values())))

7842

In [78]:
import numpy as np 

In [81]:
features_np  = np.zeros((len(G.nodes()),7842))

In [124]:
for tup in features:

    features_np[tup[0],tup[1]] = np.int8(1)

In [125]:
X = pd.DataFrame(features_np, columns = ['band_' + str(x) for x in range(1,7843) ])

In [112]:
X.drop("target",axis=1, inplace=True)

In [130]:
output_targets = {int(x):y for x,y in output_targets.items()}

In [133]:
target_df = pd.Series(output_targets)

In [135]:
X['target'] = pd.Series(target_df)

In [136]:
X

,band_1,band_2,band_3,band_4,band_5,band_6,band_7,band_8,band_9,band_10,...,band_7834,band_7835,band_7836,band_7837,band_7838,band_7839,band_7840,band_7841,band_7842,target
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7619,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
7620,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10
7621,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
7622,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,17


In [137]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X.drop("target",axis=1),X['target'], test_size=0.2, stratify=X['target'])

In [143]:
from lightgbm import LGBMClassifier

clf = LGBMClassifier(random_state=1, )

In [144]:
clf.fit(X_train,y_train)

LGBMClassifier(random_state=1)

In [145]:
from sklearn.metrics import *

print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.92      0.87      0.89       220
           1       0.00      0.00      0.00        11
          10       0.78      0.93      0.85       261
          11       1.00      0.39      0.56        28
          12       0.00      0.00      0.00        11
          13       0.00      0.00      0.00        13
          14       0.80      0.73      0.76       114
          15       0.90      0.69      0.78        51
          16       0.80      0.47      0.59        51
          17       0.71      0.92      0.80       314
           2       1.00      0.07      0.12        15
           3       0.67      0.86      0.76       103
           4       0.00      0.00      0.00         3
           5       0.78      0.67      0.72        78
           6       0.60      0.58      0.59       131
           7       0.67      0.25      0.36        16
           8       0.89      0.79      0.84        94
           9       0.00    

/Users/andrea/Documents/Ambienti_Python/miniforge3/envs/tf24/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/andrea/Documents/Ambienti_Python/miniforge3/envs/tf24/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/andrea/Documents/Ambienti_Python/miniforge3/envs/tf24/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` paramete

In [147]:
import tensorflow

In [148]:
!pip install stellargraph==1.2.1

  Using cached stellargraph-1.2.1-py3-none-any.whl (435 kB)
  Using cached gensim-4.0.1.tar.gz (23.1 MB)
ERROR: Could not find a version that satisfies the requirement tensorflow>=2.1.0 (from stellargraph)
ERROR: No matching distribution found for tensorflow>=2.1.0


In [149]:
!pip install tensorflow

ERROR: Could not find a version that satisfies the requirement tensorflow
ERROR: No matching distribution found for tensorflow


In [175]:
adj= (nx.linalg.graphmatrix.adjacency_matrix(G, nodelist=range(7264)))

In [176]:
adj=adj.todense()

In [179]:
adj.shape

(7264, 7264)

In [182]:
adj_pd = pd.DataFrame(adj)

In [183]:
adj_pd

,0,1,2,3,4,5,6,7,8,9,...,7254,7255,7256,7257,7258,7259,7260,7261,7262,7263
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7259,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7260,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7261,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7262,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [184]:
big_X = pd.concat([X,adj_pd], axis=1)

In [185]:
X_train,X_test,y_train,y_test = train_test_split(big_X.drop("target",axis=1),big_X['target'], test_size=0.2, stratify=big_X['target'])

In [189]:
clf = LGBMClassifier(random_state=1, verbose=2)

In [190]:
clf.fit(X_train,y_train)

[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.951679
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.950510
[LightGBM] [Debug] init for col-wise cost 0.245028 seconds, init for row-wise cost 0.219218 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.247261 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Sparse Multi-Val Bin
[LightGBM] [Info] Total Bins 30212
[LightGBM] [Info] Number of data points in the train set: 6099, number of used features: 15106
[LightGBM] [Info] Start training from score -1.938234
[LightGBM] [Info] Start training from score -4.954680
[LightGBM] [Info] Start training from score -1.766983
[LightGBM] [Info] Start training from score -4.015400
[LightGBM] [Info] Start training from score -4.887239
[LightGBM] [Info] Start training from score -4.803857
[LightGBM] [In

LGBMClassifier(random_state=1, verbose=2)

In [191]:
print(classification_report(y_test,clf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.92      0.88      0.90       220
           1       0.00      0.00      0.00        11
          10       0.77      0.91      0.84       261
          11       0.91      0.36      0.51        28
          12       0.00      0.00      0.00        11
          13       1.00      0.38      0.56        13
          14       0.76      0.76      0.76       114
          15       0.85      0.65      0.73        51
          16       0.70      0.41      0.52        51
          17       0.70      0.89      0.78       314
           2       0.00      0.00      0.00        15
           3       0.70      0.81      0.75       103
           4       0.00      0.00      0.00         3
           5       0.85      0.71      0.77        78
           6       0.62      0.64      0.63       131
           7       0.83      0.31      0.45        16
           8       0.91      0.74      0.82        94
           9       0.00    

/Users/andrea/Documents/Ambienti_Python/miniforge3/envs/tf24/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/andrea/Documents/Ambienti_Python/miniforge3/envs/tf24/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/andrea/Documents/Ambienti_Python/miniforge3/envs/tf24/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` paramete